In [1]:
# GPU 확인
!nvidia-smi

Thu Oct  2 04:06:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
os.environ['VLLM_USE_V1'] = '0'

In [3]:
# vLLM 설치
!pip install vllm -q

In [4]:
# 메모리 정리
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

30

In [5]:
# 모델 로딩

from vllm import LLM, SamplingParams
from transformers import AutoTokenizer, AutoModelForCausalLM
import time

model_name = "facebook/opt-125m"
print(f"\n[모델 로딩 시작] {model_name}")
print("-" * 80)

print("\n1. vLLM 모델 로드... ")
print("-" * 80)

start_vllm_load = time.time()
llm = LLM(model=model_name)
vllm_load_time = time.time() - start_vllm_load

print(f"   vLLM 로드 시간: {vllm_load_time:.2f}초")
print("-" * 80)

print("\n2. HuggingFace 모델 로드 중...")
print("-" * 80)

start_hf_load = time.time()
tokenizer = AutoTokenizer.from_pretrained(model_name)
hf_model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")
hf_load_time = time.time() - start_hf_load

print(f"   HuggingFace 로드 시간: {hf_load_time:.2f}초")
print("-" * 80)
print(f">> vLLM의 초기 로드 시간이 HuggingFace보다 긴 것이 일반적:  KV 캐시 블록 풀 메모리 사전 할당, CUDA 커널들을 사전 컴파일하고 최적화, 배치 처리 준비")
print(f">> HuggingFace는 모델만 로드하고 추론 시점에 필요한 만큼만 메모리를 할당하여 초기 로딩 속도가 더 빠름")



INFO 10-02 04:07:21 [__init__.py:216] Automatically detected platform cuda.

[모델 로딩 시작] facebook/opt-125m
--------------------------------------------------------------------------------

1. vLLM 모델 로드... 
--------------------------------------------------------------------------------
INFO 10-02 04:07:25 [utils.py:328] non-default args: {'disable_log_stats': True, 'model': 'facebook/opt-125m'}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 10-02 04:07:51 [__init__.py:742] Resolved architecture: OPTForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-02 04:07:51 [__init__.py:1815] Using max model len 2048
INFO 10-02 04:07:55 [llm_engine.py:221] Initializing a V0 LLM engine (v0.10.2) with config: model='facebook/opt-125m', speculative_config=None, tokenizer='facebook/opt-125m', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detailed_traces=None), seed=None, served_model_name=facebook/opt-125m, enable_prefix_caching=None, chunked_pref

Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 10-02 04:08:00 [default_loader.py:268] Loading weights took 1.19 seconds
INFO 10-02 04:08:01 [model_runner.py:1083] Model loading took 0.2389 GiB and 1.672711 seconds
INFO 10-02 04:08:02 [worker.py:290] Memory profiling takes 1.05 seconds
INFO 10-02 04:08:02 [worker.py:290] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 10-02 04:08:02 [worker.py:290] model weights take 0.24GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.47GiB; the rest of the memory reserved for KV Cache is 12.53GiB.
INFO 10-02 04:08:03 [executor_base.py:114] # cuda blocks: 22813, # CPU blocks: 7281
INFO 10-02 04:08:03 [executor_base.py:119] Maximum concurrency for 2048 tokens per request: 178.23x
INFO 10-02 04:08:07 [model_runner.py:1355] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 10-02 04:08:52 [model_runner.py:1507] Graph capturing finished in 44 secs, took 0.14 GiB
INFO 10-02 04:08:52 [worker.py:467] Free memory on device (14.64/14.74 GiB) on startup. Desired GPU memory utilization is (0.9, 13.27 GiB). Actual usage is 0.24 GiB for weight, 0.47 GiB for peak activation, 0.03 GiB for non-torch memory, and 0.14 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=13152064614` to fit into requested memory, or `--kv-cache-memory=14626958848` to fully utilize gpu memory. Current kv cache memory in use is 13456151654 bytes.
INFO 10-02 04:08:52 [llm_engine.py:420] init engine (profile, create kv cache, warmup model) took 50.94 seconds
INFO 10-02 04:08:52 [llm.py:295] Supported_tasks: ['generate']
INFO 10-02 04:08:52 [__init__.py:36] No IOProcessor plugins requested by the model
   vLLM 로드 시간: 86.84초
--------------------------------------------------------------------------------

2. HuggingFace 모델 로드 중...
-----------------------

In [6]:
print("1. 단일 추론 속도 비교")
print("=" * 80)

prompt = "The future of AI is"
params = SamplingParams(temperature=0.8, max_tokens=50)

# vLLM
print("\n[vLLM 추론]")
print("-" * 80)

start_vllm = time.time()
vllm_outputs = llm.generate([prompt], params)
vllm_time = time.time() - start_vllm

print(f">> 입력: {prompt}")
print(f">> 출력: {vllm_outputs[0].outputs[0].text[:100]}...")
print(f">> 처리 시간: {vllm_time:.3f}초")


# HuggingFace
print("\n[HuggingFace 추론]")
print("-" * 80)

start_hf = time.time()
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
with torch.no_grad():
    hf_outputs = hf_model.generate(
        **inputs,
        max_new_tokens=50,
        temperature=0.8,
        do_sample=True
    )
hf_time = time.time() - start_hf
hf_text = tokenizer.decode(hf_outputs[0], skip_special_tokens=True)

print(f">> 입력: {prompt}")
print(f">> 출력: {hf_text[len(prompt):len(prompt)+100]}...")
print(f">> 처리 시간: {hf_time:.3f}초")

print(f"\n>> 속도 비교: vLLM이 HuggingFace보다 {hf_time/vllm_time:.1f}배 빠름")

1. 단일 추론 속도 비교

[vLLM 추론]
--------------------------------------------------------------------------------


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

>> 입력: The future of AI is
>> 출력:  made by scientists.  If only it were true.
I think you mean "the future...".
The past is not the fu...
>> 처리 시간: 0.517초

[HuggingFace 추론]
--------------------------------------------------------------------------------
>> 입력: The future of AI is
>> 출력:  in AI and it seems that humans are starting to understand AI better than ever before.

One of the m...
>> 처리 시간: 1.576초

>> 속도 비교: vLLM이 HuggingFace보다 3.0배 빠름


In [7]:
print("2. 배치 처리 속도 비교 (Continuous Batching)")
print("=" * 80)

prompts = [
    "Artificial intelligence can",
    "Machine learning is",
    "Deep learning helps",
    "Neural networks are",
    "Python programming"
]

# vLLM
print(f"\n[vLLM 배치 추론] {len(prompts)}개 동시 처리")
print("-" * 80)

start_vllm_batch = time.time()
vllm_batch_outputs = llm.generate(prompts, params)
vllm_batch_time = time.time() - start_vllm_batch

for i, output in enumerate(vllm_batch_outputs[:5]):
    print(f"  [{i+1}] {prompts[i]}")
    print(f"      -> {output.outputs[0].text[:60]}...")

print(f"  ... (총 {len(prompts)}개)")
print(f">> 총 처리 시간: {vllm_batch_time:.3f}초")
print(f">> 처리량: {len(prompts)/vllm_batch_time:.2f} 요청/초")
print("-" * 80)

# HuggingFace
print(f"\n[HuggingFace 배치 추론] {len(prompts)}개 순차 처리")
print("-" * 80)

start_hf_batch = time.time()
hf_batch_outputs = []
for prompt_item in prompts:
    inputs = tokenizer(prompt_item, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = hf_model.generate(
            **inputs,
            max_new_tokens=50,
            temperature=0.8,
            do_sample=True
        )
    hf_batch_outputs.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
hf_batch_time = time.time() - start_hf_batch

for i in range(5):
    print(f"  [{i+1}] {prompts[i]}")
    print(f"      -> {hf_batch_outputs[i][len(prompts[i]):len(prompts[i])+60]}...")

print(f"  ... (총 {len(prompts)}개)")
print(f">> 총 처리 시간: {hf_batch_time:.3f}초")
print(f">> 처리량: {len(prompts)/hf_batch_time:.2f} 요청/초")

print(f"\n>> 배치 처리 속도 비교: vLLM이 HuggingFace보다 {hf_batch_time/vllm_batch_time:.1f}배 빠름")

2. 배치 처리 속도 비교 (Continuous Batching)

[vLLM 배치 추론] 5개 동시 처리
--------------------------------------------------------------------------------


Adding requests:   0%|          | 0/5 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/5 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

  [1] Artificial intelligence can
      ->  do all sorts of cool things.
How?
I'm a programmer and I th...
  [2] Machine learning is
      ->  a revolutionary new concept in detecting and learning thing...
  [3] Deep learning helps
      ->  with image quality

The quality of images in the SUI (Sensu...
  [4] Neural networks are
      ->  the elements of our entire human body that interact with ea...
  [5] Python programming
      ->  is used for many different programming languages, mostly in...
  ... (총 5개)
>> 총 처리 시간: 0.321초
>> 처리량: 15.57 요청/초
--------------------------------------------------------------------------------

[HuggingFace 배치 추론] 5개 순차 처리
--------------------------------------------------------------------------------
  [1] Artificial intelligence can
      ->  be a game changer, says researchers
There are thousands of ...
  [2] Machine learning is
      ->  the foundation of everything. You can't just stop learning ...
  [3] Deep learning helps
      ->  to better und

In [8]:
print("3. 메모리 효율성 비교 (PagedAttention): 긴 시퀀스 생성 시 메모리 사용량")
print("-" * 80)

test_prompt = "Write a detailed story about artificial intelligence:"
max_tokens = 2000

print(f"\n최대 생성 토큰: {max_tokens}개")
print("─" * 60)

# vLLM 메모리 측정
torch.cuda.empty_cache()
gc.collect()
torch.cuda.reset_peak_memory_stats()

memory_before_vllm = torch.cuda.memory_allocated() / (1024**2)

params = SamplingParams(temperature=0.7, max_tokens=max_tokens)
start_vllm = time.time()
vllm_output = llm.generate([test_prompt], params)
vllm_time = time.time() - start_vllm

memory_after_vllm = torch.cuda.memory_allocated() / (1024**2)
memory_peak_vllm = torch.cuda.max_memory_allocated() / (1024**2)
memory_used_vllm = memory_peak_vllm - memory_before_vllm

print(f"\nvLLM PagedAttention:")
print("-" * 80)
print(f"  처리 시간: {vllm_time:.3f}초")
print(f"  추론 전 메모리: {memory_before_vllm:.2f} MB")
print(f"  최대 메모리: {memory_peak_vllm:.2f} MB")
print(f"  추가 사용: {memory_used_vllm:.2f} MB")

# HuggingFace 메모리 측정
torch.cuda.empty_cache()
gc.collect()
torch.cuda.reset_peak_memory_stats()

memory_before_hf = torch.cuda.memory_allocated() / (1024**2)

inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")
start_hf = time.time()
with torch.no_grad():
    hf_output = hf_model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        do_sample=True
    )
hf_time = time.time() - start_hf

memory_after_hf = torch.cuda.memory_allocated() / (1024**2)
memory_peak_hf = torch.cuda.max_memory_allocated() / (1024**2)
memory_used_hf = memory_peak_hf - memory_before_hf

print(f"\nHuggingFace:")
print("-" * 80)
print(f"  처리 시간: {hf_time:.3f}초")
print(f"  추론 전 메모리: {memory_before_hf:.2f} MB")
print(f"  최대 메모리: {memory_peak_hf:.2f} MB")
print(f"  추가 사용: {memory_used_hf:.2f} MB")

# 비교
memory_savings = memory_used_hf - memory_used_vllm
memory_savings_pct = (memory_savings / memory_used_hf) * 100
speed_ratio = hf_time / vllm_time

print("\n" + "─" * 60)
print("비교 결과:")
print(f"  메모리 절약: {memory_savings:.2f} MB ({memory_savings_pct:.1f}%)")
print(f"  속도: vLLM이 {speed_ratio:.1f}배 빠름")
print("─" * 60)


print("\n vLLM PagedAttention:")
print("  - KV 캐시를 고정 크기 블록(페이지)으로 분할")
print("  - 필요할 때마다 블록을 동적으로 할당, 메모리 단편화 최소화 및 효율적 재사용")

print("\n HuggingFace:")
print("  - KV 캐시를 연속된 메모리 공간에 사전 할당, 최대 시퀀스 길이만큼 메모리 예약")

3. 메모리 효율성 비교 (PagedAttention): 긴 시퀀스 생성 시 메모리 사용량
--------------------------------------------------------------------------------

최대 생성 토큰: 2000개
────────────────────────────────────────────────────────────


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


vLLM PagedAttention:
--------------------------------------------------------------------------------
  처리 시간: 2.951초
  추론 전 메모리: 13580.13 MB
  최대 메모리: 13581.25 MB
  추가 사용: 1.12 MB

HuggingFace:
--------------------------------------------------------------------------------
  처리 시간: 37.092초
  추론 전 메모리: 13580.13 MB
  최대 메모리: 13727.83 MB
  추가 사용: 147.70 MB

────────────────────────────────────────────────────────────
비교 결과:
  메모리 절약: 146.58 MB (99.2%)
  속도: vLLM이 12.6배 빠름
────────────────────────────────────────────────────────────

 vLLM PagedAttention:
  - KV 캐시를 고정 크기 블록(페이지)으로 분할
  - 필요할 때마다 블록을 동적으로 할당, 메모리 단편화 최소화 및 효율적 재사용

 HuggingFace:
  - KV 캐시를 연속된 메모리 공간에 사전 할당, 최대 시퀀스 길이만큼 메모리 예약


In [9]:
print("4. 동시 사용자 처리 비교")
print("=" * 80)

user_queries = [f"User {i}: Hello, my name is" for i in range(1, 11)]

# vLLM 동시 처리
print(f"\n[vLLM] 10명의 동시 사용자 요청 처리 (Continuous Batching)")
print("-" * 80)
start_vllm_users = time.time()
vllm_user_outputs = llm.generate(user_queries, SamplingParams(temperature=0.7, max_tokens=20))
vllm_users_time = time.time() - start_vllm_users

for i in range(3):
    print(f"  {user_queries[i]}")
    print(f"    -> {vllm_user_outputs[i].outputs[0].text}")
print(f"  ... (총 10개 요청)")
print(f">> 처리 시간: {vllm_users_time:.3f}초")
print(f">> 처리 능력: {10/vllm_users_time:.1f} 사용자/초")
print("-" * 80)

# HuggingFace 순차 처리
print(f"\n[HuggingFace] 10명의 사용자 요청 순차 처리")
print("-" * 80)
start_hf_users = time.time()
hf_user_outputs = []
for query in user_queries:
    inputs = tokenizer(query, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = hf_model.generate(
            **inputs,
            max_new_tokens=20,
            temperature=0.7,
            do_sample=True
        )
    hf_user_outputs.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
hf_users_time = time.time() - start_hf_users

for i in range(3):
    print(f"  {user_queries[i]}")
    print(f"    -> {hf_user_outputs[i][len(user_queries[i]):]}")
print(f"  ... (총 10개 요청)")
print(f">> 처리 시간: {hf_users_time:.3f}초")
print(f">> 처리 능력: {10/hf_users_time:.1f} 사용자/초")

print(f"\n동시 사용자 처리: vLLM이 HuggingFace보다 {hf_users_time/vllm_users_time:.1f}배 빠름")
print("\nvLLM의 Continuous Batching:")
print("  - 여러 사용자 요청을 효율적으로 동시 처리, 실시간 서비스에 최적화, 높은 Throughput")
print("\nHuggingFace:")
print("  - 기본적으로 순차 처리, 배치 처리도 가능하지만 vLLM만큼 최적화되지 않음")

4. 동시 사용자 처리 비교

[vLLM] 10명의 동시 사용자 요청 처리 (Continuous Batching)
--------------------------------------------------------------------------------


Adding requests:   0%|          | 0/10 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/10 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

  User 1: Hello, my name is
    ->  Riki. I'm in the middle of a rapitation, and I'm going to start using
  User 2: Hello, my name is
    ->  Ben. I am a 22 year old male. I am currently enjoying the best nights of my life
  User 3: Hello, my name is
    ->  Brandon and I am very new to this site. I am a bartender, I'm trying to get
  ... (총 10개 요청)
>> 처리 시간: 0.130초
>> 처리 능력: 76.8 사용자/초
--------------------------------------------------------------------------------

[HuggingFace] 10명의 사용자 요청 순차 처리
--------------------------------------------------------------------------------
  User 1: Hello, my name is
    ->  Tom. I'm a new person. I'm a very nice guy. I'm a really good
  User 2: Hello, my name is
    ->  John and I am currently a graduate student at the University of Florida. I'm a member of the
  User 3: Hello, my name is
    ->  Jaden and I have a friend who I’ve been dating for about a year now.
  ... (총 10개 요청)
>> 처리 시간: 1.739초
>> 처리 능력: 5.8 사용자/초

동시 사용자 처리: vLLM이 HuggingFace